<a href="https://colab.research.google.com/github/BryanCoboF/optimizacionCosecha/blob/main/sitio/optCosechaVolCosDens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuración Inicial
Llamada a librerias

In [1]:
#Configuracion de librerias
%config IPCompleter.greedy=True
!pip install pulp
import pandas as pd
import numpy as np
import xlrd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpMinimize
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 25.3 MB/s eta 0:00:00


Carga de datos de Excel desde drive

In [12]:

from google.colab import files
from google.colab import drive

drive.mount('/content/drive') # Permite ingresar a drive desde las carpetas de drive
df = pd.read_excel("/content/drive/MyDrive/PB/BaseOpt2025Sim.xlsx") # Carga el excel de datos de proyeccion, si se necesita cambiar el archivo se debe modificar el link


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ingreso de datos deseados


1.   Volumen deseado por cada mes
3.   Densidad deseada por cada mes
2.   Edad minima y maxima deseada


In [13]:
# Volumen deseado cada mes
vecVolDeseado = []
vecVolR11minDeseado = []
vecVolR11maxDeseado = []
vecVolmaxDeseado = []
vecDensminDeseada = []
vecDensmaxDeseada = []

semanasMes = [4,4,5,4,5,4,4,5,4,4,5,4] #Numero de semanas por cada mes

#Ingreso de volumen anual deseado
#volDes = float(input("Ingrese el volumen deseado 2024: "))
volDes =35100000
# Calculo del volumen deseado para cada mes en base al deseado anual y las semanas que tiene cada mes.
for i in range(12):
  vecVolDeseado.append((volDes/52)*semanasMes[i])

# --------------------------------------------------------------------------------------------------
# Solo usado para 2023 comentar para los siguientes años
#vecVolDeseado = [2700000,2700000,3375000,2700000,3375000,2700000,2700000,3375000,2700000,2700000,3375000,2700000] #Numero de semanas por cada mes
# ---------------------------------------------------------------------------------------------------

print(vecVolDeseado)
print(sum(vecVolDeseado))

#Ingreso de edad deseada
edadMinDes = float(input("Ingrese la edad minima de cosecha : "))
areaMaxCosecha = float(input("Ingrese el area máxima cosechable por sitio cada mes: "))

[2700000.0, 2700000.0, 3375000.0, 2700000.0, 3375000.0, 2700000.0, 2700000.0, 3375000.0, 2700000.0, 2700000.0, 3375000.0, 2700000.0]
35100000.0
Ingrese la edad minima de cosecha : 26
Ingrese el area máxima cosechable por sitio cada mes: 150


OBTENCION  DE DATOS DEL EXCEL
Creación de Diccionarios Y VECTORES


In [14]:
# Obtencion de datos de sitio-año de plantacion
colSitioAnio = df['Sitio-AñoPlantación']

# Obtencion de datos area neta
colAreaNeta = df['Sum of Área Neta del Lote (Ha)']
colAreaPlant = df['Sum of Área Total del Lote (Ha)']

# ------------- Obtencion de datos de edad -----------------
# Recibe data frame con los datos de cada mes del excel
# Crea un diccionario por cada mes con los datos por sitio-lote
#Enero
colEdadEne = df['Sum of Edad pond ENE']
#Febrero
colEdadFeb = df['Sum of Edad pond FEB']
#Marzo
colEdadMar = df['Sum of Edad pond MAR']
#Abril
colEdadAbr = df['Sum of Edad pond ABR']
#Mayo
colEdadMay = df['Sum of Edad pond MAY']
#Junio
colEdadJun = df['Sum of Edad pond JUN']
#Julio
colEdadJul = df['Sum of Edad pond JUL']
#Agosto
colEdadAgo = df['Sum of Edad pond AGO']
#Septiembre
colEdadSep = df['Sum of Edad pond SEP']
#Octubre
colEdadOct = df['Sum of Edad pond OCT']
#Noviembre
colEdadNov = df['Sum of Edad pond NOV']
#Diciembre
colEdadDic = df['Sum of Edad pond DIC']


#--------OBTENCION DE DATOS DE RENDIMIENTO ----------------
# Obtiene los datos de rendimiento del data frame de excel de cada mes proyectado
#Enero
colRendProyEne = df['Sum of Rend pond ENE']
#Febrero
colRendProyFeb = df['Sum of Rend pond FEB']
#Marzo
colRendProyMar = df['Sum of Rend pond MAR']
#Abril
colRendProyAbr = df['Sum of Rend pond ABR']
#Mayo
colRendProyMay = df['Sum of Rend pond MAY']
#Junio
colRendProyJun = df['Sum of Rend pond JUN']
#Julio
colRendProyJul = df['Sum of Rend pond JUL']
#Agosto
colRendProyAgo = df['Sum of Rend pond AGO']
#Septiembre
colRendProySep = df['Sum of Rend pond SEP']
#Octubre
colRendProyOct = df['Sum of Rend pond OCT']
#Noviembre
colRendProyNov = df['Sum of Rend pond NOV']
#Dicimebre
colRendProyDic = df['Sum of Rend pond DIC']


#------- OBTENCION DE DATOS DE ACCESIBILIDAD -------------------
#Enero
colAccesoEne = df['Sum of AccesibilidadENE']
#Febrero
colAccesoFeb = df['Sum of AccesibilidadFEB']
#Marzo
colAccesoMar = df['Sum of AccesibilidadMAR']
#Abril
colAccesoAbr = df['Sum of AccesibilidadABR']
#Mayo
colAccesoMay = df['Sum of AccesibilidadMAY']
#Junio
colAccesoJun = df['Sum of AccesibilidadJUN']
#Julio
colAccesoJul = df['Sum of AccesibilidadJUL']
#Agosto
colAccesoAgo = df['Sum of AccesibilidadAGO']
#Septiembre
colAccesoSep = df['Sum of AccesibilidadSEP']
#Octubre
colAccesoOct = df['Sum of AccesibilidadOCT']
#Novimebre
colAccesoNov = df['Sum of AccesibilidadNOV']
#Diciembre
colAccesoDic = df['Sum of AccesibilidadDIC']

# Obtencion de costo de activo biologico
colCostoActBio = df['Sum of CostoActBio Pond']


# -----------OBTENCION DE COSTO OPERACIONAL ------------------------
#Enero
colCostoOpeEne = df['Sum of Costo Operacional pond ENE']
#Febrero
colCostoOpeFeb = df['Sum of Costo Operacional pond FEB']
#Marzo
colCostoOpeMar = df['Sum of Costo Operacional pond MAR']
#Abril
colCostoOpeAbr = df['Sum of Costo Operacional pond ABR']
#Mayo
colCostoOpeMay = df['Sum of Costo Operacional pond MAY']
#Junio
colCostoOpeJun = df['Sum of Costo Operacional pond JUN']
#Julio
colCostoOpeJul = df['Sum of Costo Operacional pond JUL']
#Agosto
colCostoOpeAgo = df['Sum of Costo Operacional pond AGO']
#Septiembre
colCostoOpeSep = df['Sum of Costo Operacional pond SEP']
#Octubre
colCostoOpeOct = df['Sum of Costo Operacional pond OCT']
#Noviembre
colCostoOpeNov = df['Sum of Costo Operacional pond NOV']
#Diciembre
colCostoOpeDic = df['Sum of Costo Operacional pond DIC']


#----------- OBTENCION de %R8------------------------
#Enero
colPorcR8Ene = df['Sum of %R8 pond ENE']
#Febrero
colPorcR8Feb = df['Sum of %R8 pond FEB']
#Marzo
colPorcR8Mar = df['Sum of %R8 pond MAR']
#Abril
colPorcR8Abr = df['Sum of %R8 pond ABR']
#Mayo
colPorcR8May = df['Sum of %R8 pond MAY']
#Junio
colPorcR8Jun = df['Sum of %R8 pond JUN']
#Julio
colPorcR8Jul = df['Sum of %R8 pond JUL']
#Agosto
colPorcR8Ago = df['Sum of %R8 pond AGO']
#Septiembre
colPorcR8Sep = df['Sum of %R8 pond SEP']
#Octubre
colPorcR8Oct = df['Sum of %R8 pond OCT']
#Noviembre
colPorcR8Nov = df['Sum of %R8 pond NOV']
#Diciembre
colPorcR8Dic = df['Sum of %R8 pond DIC']

# ------------OBTENCION DE PROM R8 ------------------
#Enero
colPromR8Ene = df['Sum of Prom R8 pond ENE']
#Febrero
colPromR8Feb = df['Sum of Prom R8 pond FEB']
#Marzo
colPromR8Mar = df['Sum of Prom R8 pond MAR']
#Abril
colPromR8Abr = df['Sum of Prom R8 pond ABR']
#Mayo
colPromR8May = df['Sum of Prom R8 pond MAY']
#Junio
colPromR8Jun = df['Sum of Prom R8 pond JUN']
#Julio
colPromR8Jul = df['Sum of Prom R8 pond JUL']
#Agosto
colPromR8Ago = df['Sum of Prom R8 pond AGO']
#Septiembre
colPromR8Sep = df['Sum of Prom R8 pond SEP']
#Octubre
colPromR8Oct = df['Sum of Prom R8 pond OCT']
#Noviembre
colPromR8Nov = df['Sum of Prom R8 pond NOV']
#Diciembre
colPromR8Dic = df['Sum of Prom R8 pond DIC']

#----------- OBTENCION de %R11------------------------
#Enero
colPorcR11Ene = df['Sum of %R11 pond ENE']
#Febrero
colPorcR11Feb = df['Sum of %R11 pond FEB']
#Marzo
colPorcR11Mar = df['Sum of %R11 pond MAR']
#Abril
colPorcR11Abr = df['Sum of %R11 pond ABR']
#Mayo
colPorcR11May = df['Sum of %R11 pond MAY']
#Junio
colPorcR11Jun = df['Sum of %R11 pond JUN']
#Julio
colPorcR11Jul = df['Sum of %R11 pond JUL']
#Agosto
colPorcR11Ago = df['Sum of %R11 pond AGO']
#Septiembre
colPorcR11Sep = df['Sum of %R11 pond SEP']
#Octubre
colPorcR11Oct = df['Sum of %R11 pond OCT']
#Noviembre
colPorcR11Nov = df['Sum of %R11 pond NOV']
#Diciembre
colPorcR11Dic = df['Sum of %R11 pond DIC']


# ------------OBTENCION DE PROM R11 ------------------
#Enero
colPromR11Ene = df['Sum of Prom R11 pond ENE']
#Febrero
colPromR11Feb = df['Sum of Prom R11 pond FEB']
#Marzo
colPromR11Mar = df['Sum of Prom R11 pond MAR']
#Abril
colPromR11Abr = df['Sum of Prom R11 pond ABR']
#Mayo
colPromR11May = df['Sum of Prom R11 pond MAY']
#Junio
colPromR11Jun = df['Sum of Prom R11 pond JUN']
#Julio
colPromR11Jul = df['Sum of Prom R11 pond JUL']
#Agosto
colPromR11Ago = df['Sum of Prom R11 pond AGO']
#Septiembre
colPromR11Sep = df['Sum of Prom R11 pond SEP']
#Octubre
colPromR11Oct = df['Sum of Prom R11 pond OCT']
#Noviembre
colPromR11Nov = df['Sum of Prom R11 pond NOV']
#Diciembre
colPromR11Dic = df['Sum of Prom R11 pond DIC']



# ---------CREACION DE MATRICES POR CADA MES DEL AÑO ------------
filas = len(colSitioAnio)  # Número de filas
columnas = 12  # Número de columnas (meses del siguiente año)


#//MATRICES//
# Inicializa las matrices n filas y 12 columnas
#Todos los valores se inicializan  con cero  /// Todas las matrices son del mismo tamaño
matrizEdad = [[0] * columnas for _ in range(filas)]
matrizRend = [[0] * columnas for _ in range(filas)]
matrizRendAux = [[0] * columnas for _ in range(filas)]
matrizAcceso = [[0] * columnas for _ in range(filas)]
matrizCostoOpe = [[0] * columnas for _ in range(filas)]
matrizCosto = [[0] * columnas for _ in range(filas)]
matrizRendPlant = [[0] * columnas for _ in range(filas)]  #Rendimiento area plantada
matrizPorcR8 = [[0] * columnas for _ in range(filas)]
matrizPromR8 = [[0] * columnas for _ in range(filas)]
matrizPorcR11 = [[0] * columnas for _ in range(filas)]
matrizPromR11 = [[0] * columnas for _ in range(filas)]


# -------------Asignacion de valores a cada mes de las matrices [n][12]---------
  # Matriz de Edades por cada mes del 2024
  # Matriz  de rendimientos
  # Matriz de Accesibilidad
  # Matriz de rendimiento de area plantada
  # Matriz de costo operacional
  # Matriz de porcentaje R8
  # Matriz de promedio R8
  # Matriz de porcentaje R11
  # Matriz de promedio R11
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    if i == 0:
      matrizEdad[j][i] = colEdadEne[j]
      matrizRend[j][i] = colRendProyEne[j] #Rendimiento neto
      matrizAcceso[j][i] = colAccesoEne [j]
      matrizRendPlant[j][i] = colRendProyEne[j]*colAreaNeta[j]/colAreaPlant[j] #Rendimiento area plantada
      matrizCostoOpe[j][i] = colCostoOpeEne[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Ene[j]
      matrizPromR8[j][i] = colPromR8Ene[j]
      matrizPorcR11[j][i] = colPorcR11Ene[j]
      matrizPromR11[j][i] = colPromR11Ene[j]


    if i == 1:
      matrizEdad[j][i] = colEdadFeb[j]
      matrizRend[j][i] = colRendProyFeb[j]
      matrizAcceso[j][i] = colAccesoFeb [j]
      matrizRendPlant[j][i] = colRendProyFeb[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeFeb[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Feb[j]
      matrizPromR8[j][i] = colPromR8Feb[j]
      matrizPorcR11[j][i] = colPorcR11Feb[j]
      matrizPromR11[j][i] = colPromR11Feb[j]


    if i == 2:
      matrizEdad[j][i] = colEdadMar[j]
      matrizRend[j][i] = colRendProyMar[j]
      matrizAcceso[j][i] = colAccesoMar [j]
      matrizRendPlant[j][i] = colRendProyMar[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeMar[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Mar[j]
      matrizPromR8[j][i] = colPromR8Mar[j]
      matrizPorcR11[j][i] = colPorcR11Mar[j]
      matrizPromR11[j][i] = colPromR11Mar[j]


    if i == 3:
      matrizEdad[j][i] = colEdadAbr[j]
      matrizRend[j][i] = colRendProyAbr[j]
      matrizAcceso[j][i] = colAccesoAbr [j]
      matrizRendPlant[j][i] = colRendProyAbr[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeAbr[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Abr[j]
      matrizPromR8[j][i] = colPromR8Abr[j]
      matrizPorcR11[j][i] = colPorcR11Abr[j]
      matrizPromR11[j][i] = colPromR11Abr[j]


    if i == 4:
      matrizEdad[j][i] = colEdadMay[j]
      matrizRend[j][i] = colRendProyMay[j]
      matrizAcceso[j][i] = colAccesoMay [j]
      matrizRendPlant[j][i] = colRendProyMay[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeMay[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8May[j]
      matrizPromR8[j][i] = colPromR8May[j]
      matrizPorcR11[j][i] = colPorcR11May[j]
      matrizPromR11[j][i] = colPromR11May[j]


    if i == 5:
      matrizEdad[j][i] = colEdadJun[j]
      matrizRend[j][i] = colRendProyJun[j]
      matrizAcceso[j][i] = colAccesoJun [j]
      matrizRendPlant[j][i] = colRendProyJun[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeJun[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Jun[j]
      matrizPromR8[j][i] = colPromR8Jun[j]
      matrizPorcR11[j][i] = colPorcR11Jun[j]
      matrizPromR11[j][i] = colPromR11Jun[j]

    if i == 6:
      matrizEdad[j][i] = colEdadJul[j]
      matrizRend[j][i] = colRendProyJul[j]
      matrizAcceso[j][i] = colAccesoJul [j]
      matrizRendPlant[j][i] = colRendProyJul[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeJul[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Jul[j]
      matrizPromR8[j][i] = colPromR8Jul[j]
      matrizPorcR11[j][i] = colPorcR11Jul[j]
      matrizPromR11[j][i] = colPromR11Jul[j]

    if i == 7:
      matrizEdad[j][i] = colEdadAgo[j]
      matrizRend[j][i] = colRendProyAgo[j]
      matrizAcceso[j][i] = colAccesoAgo [j]
      matrizRendPlant[j][i] = colRendProyAgo[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeAgo[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Ago[j]
      matrizPromR8[j][i] = colPromR8Ago[j]
      matrizPorcR11[j][i] = colPorcR11Ago[j]
      matrizPromR11[j][i] = colPromR11Ago[j]


    if i == 8:
      matrizEdad[j][i] = colEdadSep[j]
      matrizRend[j][i] = colRendProySep[j]
      matrizAcceso[j][i] = colAccesoSep [j]
      matrizRendPlant[j][i] = colRendProySep[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeSep[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Sep[j]
      matrizPromR8[j][i] = colPromR8Sep[j]
      matrizPorcR11[j][i] = colPorcR11Sep[j]
      matrizPromR11[j][i] = colPromR11Sep[j]


    if i == 9:
      matrizEdad[j][i] = colEdadOct[j]
      matrizRend[j][i] = colRendProyOct[j]
      matrizAcceso[j][i] = colAccesoOct [j]
      matrizRendPlant[j][i] = colRendProyOct[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeOct[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Oct[j]
      matrizPromR8[j][i] = colPromR8Oct[j]
      matrizPorcR11[j][i] = colPorcR11Oct[j]
      matrizPromR11[j][i] = colPromR11Oct[j]


    if i == 10:
      matrizEdad[j][i] = colEdadNov[j]
      matrizRend[j][i] = colRendProyNov[j]
      matrizAcceso[j][i] = colAccesoNov [j]
      matrizRendPlant[j][i] = colRendProyNov[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeNov[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Nov[j]
      matrizPromR8[j][i] = colPromR8Nov[j]
      matrizPorcR11[j][i] = colPorcR11Nov[j]
      matrizPromR11[j][i] = colPromR11Nov[j]


    if i == 11:
      matrizEdad[j][i] = colEdadDic[j]
      matrizRend[j][i] = colRendProyDic[j]
      matrizAcceso[j][i] = colAccesoDic [j]
      matrizRendPlant[j][i] = colRendProyDic[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeDic[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Dic[j]
      matrizPromR8[j][i] = colPromR8Dic[j]
      matrizPorcR11[j][i] = colPorcR11Dic[j]
      matrizPromR11[j][i] = colPromR11Dic[j]



Creacion del diccionario con todas las edades

RESTRICCION DE EDAD DE COSECHA Y ACCESIBILIDAD

In [15]:
# Si el sitio cumple con los requerimientos de edad y accesibilidad en ese mes se asgina el rendimiento proyectado,
# caso contrario se asigna 0
# Pregunta primero si desea aplicar las restricciones

restEdad = float(input("Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : "))
restAcceso = float(input("Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : "))


if restEdad == 1 and restAcceso == 1:
  # Restricciones completas --- Se corre solo si se ingreso que se desea restringir edad y acceso
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1 and matrizEdad[j][i] >= 26:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 1:
  # Restriccion de acceso unicamente
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 1 and restAcceso == 0:
  # Restriccion solo de edad
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizEdad[j][i] >= 26:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 0:
  # Sin restricciones
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      matrizRendAux[j][i] = matrizRend[j][i]

Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : 1
Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : 0


-------------------------------------------

MODELO OPTIMIZACION

------------------------------------------------------------

In [56]:
from pandas.core.internals.ops import operate_blockwise

#****************************** MODELO DE OPTIMIZACION *********************************************************************************
#-------------  Definicion del modelo   -----------------
optCosecha = pulp.LpProblem('OptimizacionCosecha',pulp.LpMaximize) #Inicialmente busca minimizar el area de cosecha para obtener el volumen necesario para Enero

# ------------VARIABLE DE DECISION (AREA A COSECHAR)-------------------
area_a_cosechar = [[pulp.LpVariable(f'x{i}_{j}', lowBound=0, cat='Continuous') for j in range(columnas)] for i in range(filas)]

#--------------FUNCION OBJETIVO-------------------------
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas))

#--------------RESTRICCIONES----------------------------
# 1) -- Restringe las areas a cosechar de cada sitio y mes para que no sean mayores al area neta
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    optCosecha += area_a_cosechar[j][i] >= 0
    optCosecha += area_a_cosechar[j][i] <= colAreaNeta[j]
    optCosecha += area_a_cosechar[j][i] <= areaMaxCosecha

# 2) -- Restringe que la suma del area a cosecha de cada sitio en todo el año sea menor o igual al area neta
for j in range(filas): # recorre filas
  optCosecha += sum(area_a_cosechar[j]) <= colAreaNeta[j]

# 3) Restriccion de volumen anual
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas)) <= volDes

# 4) Restringe los volumenes de cada mes dependiendo del volumen deseado
for j in range(columnas):
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= vecVolDeseado[j]
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= vecVolDeseado[j]-50000

# 5) Restringe los costos
for j in range(columnas):
  # SOLO COSTO DE ACTIVO BIOLOGICO
  optCosecha += sum(((colCostoActBio[i]*area_a_cosechar[i][j]))/(matrizRendAux[i][j]*colAreaNeta[i]/1000) for i in range(filas) if matrizRendAux[i][j]>0) >=80
  optCosecha += sum(((colCostoActBio[i]*area_a_cosechar[i][j]))/(matrizRendAux[i][j]*colAreaNeta[i]/1000) for i in range(filas) if matrizRendAux[i][j]>0) <=150
  #Calculo de costos resultantes
  # INCLUYE COSTO OPERACIONAL DE COSECHA
  #optCosecha += sum(((colCostoActBio[i]*area_a_cosechar[i][j])+matrizCostoOpe[i][j])/(matrizRendAux[i][j]*colAreaNeta[i]/1000) for i in range(filas) if matrizRendAux[i][j]>0) >=100
  #optCosecha += sum(((colCostoActBio[i]*area_a_cosechar[i][j])+matrizCostoOpe[i][j])/(matrizRendAux[i][j]*colAreaNeta[i]/1000) for i in range(filas) if matrizRendAux[i][j]>0) <=180

# 6) Restriccion de promedio densidad R8 y R11
for j in range(columnas):
  optCosecha += sum(matrizPromR8[i][j]*area_a_cosechar[i][j] for i in range(filas)) >=7.5*sum(area_a_cosechar[i][j] for i in range(filas))
  optCosecha += sum(matrizPromR8[i][j]*area_a_cosechar[i][j] for i in range(filas)) <=8.5*sum(area_a_cosechar[i][j] for i in range(filas))
  optCosecha += sum(matrizPromR11[i][j]*area_a_cosechar[i][j] for i in range(filas))>=5.5*sum(area_a_cosechar[i][j] for i in range(filas))
  optCosecha += sum(matrizPromR11[i][j]*area_a_cosechar[i][j] for i in range(filas))<=6.5*sum(area_a_cosechar[i][j] for i in range(filas))

'''
#RESTRICCION DE RENDIMIENTO
#Restringe area de ene para rendimiento de 11500     (2.7 MILL con 234Ha)   (3.375 MILL con 293Ha)
optCosecha += sum(area_a_cosechar[i][0] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][0] for i in range(filas)) <= 280
#Restringe area de feb para rendimiento de
optCosecha += sum(area_a_cosechar[i][1] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][1] for i in range(filas)) <= 280
#Restringe area de mar para rendimiento de
optCosecha += sum(area_a_cosechar[i][2] for i in range(filas)) >= 270
optCosecha += sum(area_a_cosechar[i][2] for i in range(filas)) <= 340
#Restringe area de abr para rendimiento de
optCosecha += sum(area_a_cosechar[i][3] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][3] for i in range(filas)) <= 280
#Restringe area de may para rendimiento de
optCosecha += sum(area_a_cosechar[i][4] for i in range(filas)) >= 270
optCosecha += sum(area_a_cosechar[i][4] for i in range(filas)) <= 340
#Restringe area de jun para rendimiento de
optCosecha += sum(area_a_cosechar[i][5] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][5] for i in range(filas)) <= 280
#Restringe area de jul para rendimiento de
optCosecha += sum(area_a_cosechar[i][6] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][6] for i in range(filas)) <= 280
#Restringe area de agosto para rendimiento de
optCosecha += sum(area_a_cosechar[i][7] for i in range(filas)) >= 270
optCosecha += sum(area_a_cosechar[i][7] for i in range(filas)) <= 340
#Restringe area de sep para rendimiento de 11700
optCosecha += sum(area_a_cosechar[i][8] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][8] for i in range(filas)) <= 280
#Restringe area de OCT para rendimiento de 11700
optCosecha += sum(area_a_cosechar[i][9] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][9] for i in range(filas)) <= 280
#Restringe area de nov para rendimiento de 11700
optCosecha += sum(area_a_cosechar[i][10] for i in range(filas)) >= 270
optCosecha += sum(area_a_cosechar[i][10] for i in range(filas)) <= 340
#Restringe area de dic para rendimiento de 11700
optCosecha += sum(area_a_cosechar[i][11] for i in range(filas)) >= 210
optCosecha += sum(area_a_cosechar[i][11] for i in range(filas)) <= 280
'''


#Resolucion
optCosecha.solve()
#****************************** FIN MODELO*********************************************************************************

#--------------Impresion de resultados preliminares--------------------------------------------------------------------
a=1
for j in range(filas): # recorre filas
  # Imprime cada sitio con el area asignada en el mes que se va a cosechar
  print(a, ") Areas ENE ", colSitioAnio[j], "-> " ,round(area_a_cosechar[j][0].value(),1), ", " ,round(area_a_cosechar[j][1].value(),1), ", " ,round(area_a_cosechar[j][2].value(),1),", " ,round(area_a_cosechar[j][3].value(),1),", " ,round(area_a_cosechar[j][4].value(),1),", " ,round(area_a_cosechar[j][5].value(),1),", " ,round(area_a_cosechar[j][6].value(),1),", " ,round(area_a_cosechar[j][7].value(),1), ", " ,round(area_a_cosechar[j][8].value(),1),", " ,round(area_a_cosechar[j][9].value(),1),", " ,round(area_a_cosechar[j][10].value(),1),", " ,round(area_a_cosechar[j][11].value(),1))
  a = a+1

for b in range(columnas):
  #Imprime el volumen a entregar cada mes
  print("Vol> ", b+1, "->", round(sum(matrizRendAux[i][b]*area_a_cosechar[i][b].value() for i in range(filas) ),0))

#Imprime el volumen total anual
print(sum(matrizRendAux[i][j]*area_a_cosechar[i][j].value() for j in range(columnas) for i in range(filas) ))
#-----------------------------------------------------------------------------------------------------------------------





1 ) Areas ENE  CEB-042022 ->  0.0 ,  0.0 ,  0.0 ,  27.4 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
2 ) Areas ENE  FC-012022 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  28.7 ,  0.0 ,  0.0
3 ) Areas ENE  FC-032021 ->  0.0 ,  0.0 ,  18.5 ,  6.7 ,  5.6 ,  0.0 ,  0.0 ,  0.0 ,  7.1 ,  0.0 ,  0.0 ,  0.0
4 ) Areas ENE  FC-042021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
5 ) Areas ENE  FC-072021 ->  26.6 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
6 ) Areas ENE  FC-112021 ->  0.0 ,  35.2 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
7 ) Areas ENE  FM-112021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  2.2 ,  0.0 ,  12.4 ,  0.0 ,  0.0
8 ) Areas ENE  FM-152021 ->  31.5 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
9 ) Areas ENE  FM-202022 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
10

Correccion de errores

In [57]:
# Matriz que obtiene los resultados de las areas optimizadas de cosecha
matrizAreaRes = [[0] * columnas for _ in range(filas)]

#Asignacion de resultados de area e impresion de errores
#Impresion de errores
countErrores = 0
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    matrizAreaRes [j][i] = area_a_cosechar[j][i].value()
    if area_a_cosechar[j][i].value() <0:
      countErrores = countErrores + 1
print("Errores-> ", countErrores)


#Vectores de resultados por mes
volTotalRes = []
areaTotalRes = []
rendTotalRes = []
costoTotalRes = []
volTotalR8Res = []
volTotalR11Res = []
promR8Res = []
promR11Res = []

#------------CALCULO DE RESULTADOS POR MES -----------------------
#calculo de costos resultantes por mes
for j in range(columnas):
  #costoTotalRes1 = sum(((colCostoActBio[i]*matrizAreaRes[i][j])+matrizCostoOpe[i][j])/(matrizRendAux[i][j]*colAreaNeta[i]/1000) for i in range(filas) if matrizRendAux[i][j]>0)
  costoTotalRes1 = sum(((colCostoActBio[i]*matrizAreaRes[i][j]))/(matrizRendAux[i][j]*colAreaNeta[i]/1000) for i in range(filas) if matrizRendAux[i][j]>0)
  costoTotalRes.append(round(costoTotalRes1,1))


#Calculo de volumenes (total, R8, y R11) y area resultantes por mes
for j in range(columnas):
  # Volumen total por mes
  volTotal = sum(matrizRendAux[i][j]*matrizAreaRes[i][j] for i in range(filas))
  volTotalRes.append(round(volTotal,0))
  #Area Total por mes
  areaTotal = sum(matrizAreaRes[i][j] for i in range(filas))
  areaTotalRes.append(round(areaTotal,1))
  #Volumen R8 total por mes
  volTotalR8 = sum(matrizRendAux[i][j]*matrizAreaRes[i][j]*matrizPorcR8[i][j] for i in range(filas))
  volTotalR8Res.append(round(volTotalR8,0))
  #Volumen R11 total por mes
  volTotalR11 = sum(matrizRendAux[i][j]*matrizAreaRes[i][j]*matrizPorcR11[i][j] for i in range(filas))
  volTotalR11Res.append(round(volTotalR11,0))
  # Promedio R8 resultante por mes
  promR8 = sum(matrizAreaRes[i][j]*matrizPromR8[i][j] for i in range(filas))
  promR8Res.append(round(promR8/areaTotal,1))
  # Promedio R11 resultante por mes
  promR11 = sum(matrizAreaRes[i][j]*matrizPromR11[i][j] for i in range(filas))
  promR11Res.append(round(promR11/areaTotal,1))


#Rendimiento resultante por mes
for k in range(len(volTotalRes)):
  if volTotalRes[k]>0:
    rendTotalRes.append(round(volTotalRes[k]/areaTotalRes[k],1))
  else:
    rendTotalRes.append(0)


#Presentacion de resultados
print("----------------------------------")
print('Volumen Anual Deseado')
print(sum(vecVolDeseado))
print("Volumen anual Resultante")
print(sum(volTotalRes))
print('Volumen mensual deseado')
print(vecVolDeseado)
print("Volumen Resultante por mes")
print(volTotalRes)
print("Volumen R8 por mes")
print(volTotalR8Res)
print("Promedio R8 por mes")
print(promR8Res)
print("Volumen R11 por mes")
print(volTotalR11Res)
print("Promedio R11 por mes")
print(promR11Res)
print("Area a cosechar por mes")
print(areaTotalRes)
print("Costo kbft por mes")
print(costoTotalRes)
print("Rendimiento por mes")
print(rendTotalRes)
print("----------------------------------")


# Remanentes
for j in range(filas): # recorre filas
  if round(sum(matrizAreaRes[j]),2)<= round(colAreaNeta[j],2)-1:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2), "-- Remanente 2025 -- ",round(colAreaNeta[j],2)-round(sum(matrizAreaRes[j]),2))
  else:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2))



Errores->  0
----------------------------------
Volumen Anual Deseado
35100000.0
Volumen anual Resultante
35100000.0
Volumen mensual deseado
[2700000.0, 2700000.0, 3375000.0, 2700000.0, 3375000.0, 2700000.0, 2700000.0, 3375000.0, 2700000.0, 2700000.0, 3375000.0, 2700000.0]
Volumen Resultante por mes
[2700000.0, 2700000.0, 3375000.0, 2700000.0, 3375000.0, 2700000.0, 2700000.0, 3375000.0, 2700000.0, 2700000.0, 3375000.0, 2700000.0]
Volumen R8 por mes
[622114.0, 723505.0, 994195.0, 791835.0, 1531831.0, 1229140.0, 1205460.0, 1507880.0, 1200605.0, 1179571.0, 1504089.0, 1213713.0]
Promedio R8 por mes
[7.6, 7.7, 7.7, 7.6, 8.5, 8.5, 8.5, 8.5, 8.5, 8.0, 8.4, 8.5]
Volumen R11 por mes
[2077826.0, 1976428.0, 2380711.0, 1908087.0, 1842715.0, 1470516.0, 1494224.0, 1866701.0, 1499086.0, 1520209.0, 1870599.0, 1485990.0]
Promedio R11 por mes
[5.5, 5.7, 5.7, 5.6, 6.4, 6.4, 6.4, 6.4, 6.4, 6.0, 6.4, 6.4]
Area a cosechar por mes
[271.3, 236.1, 272.5, 256.8, 181.7, 123.2, 147.6, 225.8, 195.1, 324.6, 201.1, 

Creacion del Excel con resultados


In [58]:
columnasRes = 13+12+12
matrizResultados = [[0] * columnasRes for _ in range(filas)]

#Asignación de nombres de sitio a la matriz de resultados
for h in range(filas):
  matrizResultados[h][0] = colSitioAnio[h]
k=1

# Asignación de resultados a la matriz
for k in range(columnasRes):
  for h in range(filas):
    if k>0 and k<=12:
      matrizResultados[h][k] = area_a_cosechar[h][k-1].value()
    if k>12 and k<=24:
      matrizResultados[h][k] = matrizRendAux[h][k-1-12]
    if k>24:
      matrizResultados[h][k] = matrizRendAux[h][k-1-24]*area_a_cosechar[h][k-1-24].value()

for f in range(filas):
  print(matrizResultados[f])

# Crea el Excel con los resultados,  areas por mes
#Nombres de las columnas del excel
nomColRes = ['Sitio-Año de plantación', 'Área ENE 2024', 'Área FEB 2024','Área MAR 2024', 'Área ABR 2024','Área MAY 2024','Área JUN 2024','Área JUL 2024','Área AGO 2024', 'Área SEP 2024','Área OCT 2024','Área NOV 2024','Área DIC 2024','Rendimiento ENE 2024','Rendimiento FEB 2024','Rendimiento MAR 2024','Rendimiento ABR 2024','Rendimiento MAY 2024','Rendimiento JUN 2024','Rendimiento JUL 2024','Rendimiento AGO 2024','Rendimiento SEP 2024','Rendimiento OCT 2024','Rendimiento NOV 2024','Rendimiento DIC 2024','VOL ENE 2024','VOL FEB 2024','VOL MAR 2024','VOL ABR 2024','VOL MAY 2024','VOL JUN 2024','VOL JUL 2024','VOL AGO 2024','VOL SEP 2024','VOL OCT 2024','VOL NOV 2024','VOL DIC 2024']
#Crea el data frame con los nombres de columnas y la matriz de resultados
df1 = pd.DataFrame(matrizResultados, columns=nomColRes)
# Path donde se crea el archivo resOptimizacion
pathExcel = '/content/drive/MyDrive/PB/resOptimizacion.xlsx'
#Guarda el data frame en el excel
df1.to_excel(pathExcel, index=False)


#Creacion de matriz de remanentes.
colRem = 4
matrizRemanentes = [[0] * colRem for _ in range(filas)]
for j in range(filas):
  matrizRemanentes[j][0] = colSitioAnio[j]
  matrizRemanentes[j][1] = sum(matrizAreaRes[j])
  matrizRemanentes[j][2] = colAreaNeta[j]
  matrizRemanentes[j][3] = colAreaNeta[j] - sum(matrizAreaRes[j])

print(matrizRemanentes)
#Titulos de remanentes
nomMatRem = ['Sitio-Año de plantación','Area a cosechar', 'Area neta Sitio', 'Remanente']
#Crea el data frame de remanentes
df2 =pd.DataFrame(matrizRemanentes, columns=nomMatRem)
pathExcel2 = '/content/drive/MyDrive/PB/remanenteOptimizacion.xlsx'
#Guarda el data frame en el excel
df2.to_excel(pathExcel2, index=False)


['CEB-042022', 0.0, 0.0, 0.0, 27.43, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12253.77965761613, 12706.29593185864, 13125.98409420765, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['FC-012022', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 28.66, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15056.098823043354, 15588.748682840636, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['FC-032021', 0.0, 0.0, 18.505958, 6.7075757, 5.5767652, 0.0, 0.0, 0.0, 7.1297007, 0.0, 0.0, 0.0, 15789.131392027963, 16463.252957986544, 17089.450203568576, 17671.22990610113, 18212.008167789332, 18715.032320803395, 19183.33721977812, 19619.72554802513, 20026.76427433133, 20406.791467029252, 20761.929310519856, 21094.10042587064, 0.0, 0.0, 316256.64771033154, 118531.1123072772, 101564.0933722433, 0.0, 0.0, 0.0, 142784.83526543507, 0.0, 0.0, 0.0]
['FC-042021', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12481.196272996332, 13163.728824645188, 13802.81